In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.layers import Dense,LSTM,RepeatVector, TimeDistributed
from keras.models import Sequential,Model
from keras import optimizers
from sklearn.metrics import mean_squared_error
from keras import backend as K

Using TensorFlow backend.


In [0]:
from google.colab import files
uploaded = files.upload()

Saving data-series.csv to data-series (1).csv


In [0]:
labels=files.upload()

In [0]:
from io import StringIO

s=str(uploaded['data-series.csv'],'utf-8')

data = StringIO(s) 

dataseries=pd.read_csv(data)
dataseries=dataseries.drop(['Unnamed: 0'],axis=1)
dataseries.head(20)

,0,1,2,3,4,5,6,7,8,9,10,11
0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0
4,2,2,0,2,0,0,1,0,0,0,0,1
5,2,0,2,0,0,1,0,0,0,0,1,0
6,1,0,0,0,0,1,0,1,1,0,1,0
7,0,0,0,0,1,0,1,1,0,1,0,1
8,0,0,0,0,0,1,0,0,0,0,0,0
9,0,0,0,0,1,0,0,0,0,0,0,0


In [0]:
from io import StringIO

s1=str(labels['labels-series.csv'],'utf-8')

data1 = StringIO(s1) 

labels=pd.read_csv(data1,names=['index','labels'])
labels.head(20)


,index,labels
0,0,0
1,1,0
2,2,0
3,3,1
4,4,0
5,5,0
6,6,1
7,7,0
8,8,0
9,9,0


In [0]:
labels=labels.drop(['index'],axis=1)

In [0]:
from sklearn.model_selection import train_test_split
train, valid, Y_train, Y_valid = train_test_split(dataseries, labels.values, test_size=0.10, random_state=0)
X_train = train.values.reshape((train.shape[0], train.shape[1], 1))
X_valid = valid.values.reshape((valid.shape[0], valid.shape[1], 1))
print("Train set reshaped", Y_train.shape)
print("Validation set reshaped", Y_valid.shape)
Y_train

Train set reshaped (200327, 1)
Validation set reshaped (22259, 1)


array([[0],
       [0],
       [1],
       ...,
       [6],
       [0],
       [0]])

In [0]:
serie_size =  X_train.shape[1] # 12
n_features =  X_train.shape[2] # 1
lstm_model = Sequential()
lstm_model.add(LSTM(10, input_shape=(serie_size, n_features), return_sequences=True))
lstm_model.add(LSTM(6, activation='relu', return_sequences=True))
lstm_model.add(LSTM(1, activation='relu'))
lstm_model.add(Dense(10, kernel_initializer='glorot_normal', activation='relu'))
lstm_model.add(Dense(10, kernel_initializer='glorot_normal', activation='relu'))
lstm_model.add(Dense(1))
adam = optimizers.Adam(0.0001)
lstm_model.compile(loss='mse', optimizer=adam)

W0705 03:55:16.674689 139960618211200 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0705 03:55:16.714749 139960618211200 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0705 03:55:16.721505 139960618211200 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0705 03:55:17.268639 139960618211200 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4185: The name tf.truncated_normal is deprecated. Please use tf.random.truncated_normal instead.

W0705 03:55:17.319633 139960618211200 de

In [0]:
lstm_history = lstm_model.fit(X_train, Y_train, epochs=20, batch_size=64, validation_data=(X_valid, Y_valid))

W0704 04:48:33.756823 140504725673856 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0704 04:48:35.920676 140504725673856 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.



Train on 200327 samples, validate on 22259 samples
Epoch 1/20
200327/200327 [==============================] - 254s 1ms/step - loss: 1.6756 - val_loss: 1.5144
Epoch 2/20
200327/200327 [==============================] - 252s 1ms/step - loss: 1.5851 - val_loss: 1.4807
Epoch 3/20
200327/200327 [==============================] - 249s 1ms/step - loss: 1.5706 - val_loss: 1.4791
Epoch 4/20
200327/200327 [==============================] - 247s 1ms/step - loss: 1.5699 - val_loss: 1.4791
Epoch 5/20
200327/200327 [==============================] - 254s 1ms/step - loss: 1.5699 - val_loss: 1.4792
Epoch 6/20
200327/200327 [==============================] - 252s 1ms/step - loss: 1.5699 - val_loss: 1.4791
Epoch 7/20
200327/200327 [==============================] - 250s 1ms/step - loss: 1.5699 - val_loss: 1.4791
Epoch 8/20
200327/200327 [==============================] - 250s 1ms/step - loss: 1.5699 - val_loss: 1.4791
Epoch 9/20
200327/200327 [==============================] - 249s 1ms/step - loss: 1.5

In [0]:
model=lstm_model
model_json = model.to_json()
with open("model_lstm.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model_lstm.h5")
print("Saved model to disk")

Saved model to disk


In [0]:
encoder_decoder = Sequential()
encoder_decoder.add(LSTM(serie_size, activation='relu', input_shape=(serie_size, n_features), return_sequences=True))
encoder_decoder.add(LSTM(6, activation='relu', return_sequences=True))
encoder_decoder.add(LSTM(1, activation='relu'))
encoder_decoder.add(RepeatVector(serie_size))
encoder_decoder.add(LSTM(serie_size, activation='relu', return_sequences=True))
encoder_decoder.add(LSTM(6, activation='relu', return_sequences=True))
encoder_decoder.add(TimeDistributed(Dense(1)))
encoder_decoder.summary()
adam = optimizers.Adam(0.0001)
def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true))) 
encoder_decoder.compile(loss='mse', optimizer=adam)

NameError: ignored

In [0]:
encoder_decoder_history = encoder_decoder.fit(X_train, X_train, epochs=20, batch_size=64)

Epoch 1/20
200327/200327 [==============================] - 135s 673us/step - loss: 1269134746.0799 - acc: 0.4272
Epoch 2/20
200327/200327 [==============================] - 127s 635us/step - loss: 1144078657.7668 - acc: 0.4389
Epoch 3/20
200327/200327 [==============================] - 129s 643us/step - loss: 3344122.7618 - acc: 0.0015
Epoch 4/20
200327/200327 [==============================] - 128s 640us/step - loss: 7242928.8772 - acc: 0.0022
Epoch 5/20
200327/200327 [==============================] - 129s 642us/step - loss: 20258.8263 - acc: 0.0032
Epoch 6/20
200327/200327 [==============================] - 130s 647us/step - loss: 4935.7277 - acc: 0.0112
Epoch 7/20
200327/200327 [==============================] - 129s 644us/step - loss: 903.4910 - acc: 0.0266
Epoch 8/20
200327/200327 [==============================] - 129s 642us/step - loss: 298.8947 - acc: 0.0635
Epoch 9/20
200327/200327 [==============================] - 129s 643us/step - loss: 421.3689 - acc: 0.1046
Epoch 10/20


In [0]:
model=encoder_decoder
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")

NameError: ignored

In [0]:
from keras.models import model_from_json
json_file = open('modelv1.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
loaded_model.load_weights("modelv1.h5")

W0705 03:55:58.051115 139960618211200 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("ls..., inputs=[<tf.Tenso...)`
  """Entry point for launching an IPython kernel.


In [0]:
json_file = open('model_lstm.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model_lstm = model_from_json(loaded_model_json)
loaded_model_lstm.load_weights("model_lstm.h5")

In [0]:
train_encoded = encoder.predict(X_train)
validation_encoded = encoder.predict(X_valid)
print('Encoded time-series shape', train_encoded.shape)
print('Encoded time-series sample', train_encoded[0])

Encoded time-series shape (200327, 1)
Encoded time-series sample [0.]


In [0]:
train['encoded'] = train_encoded
train['label'] = Y_train

valid['encoded'] = validation_encoded
valid['label'] = Y_valid

train.head(10)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/p

,0,1,2,3,4,5,6,7,8,9,10,11,encoded,label
207604,0,0,0,0,0,0,0,0,0,0,0,0,0.000000,0
45150,0,0,0,0,0,0,0,0,0,0,0,0,0.000000,0
143433,0,0,4,2,1,2,2,1,0,0,0,1,0.512935,1
202144,0,0,0,0,0,0,0,0,0,0,0,0,0.000000,0
136088,0,0,0,0,0,0,0,1,0,0,1,0,0.000000,1
121675,0,0,0,0,0,1,0,0,0,0,0,0,0.000000,0
185281,0,0,0,0,0,0,0,0,0,0,0,0,0.000000,1
70087,0,0,0,0,0,0,0,0,3,0,1,3,0.103709,0
105249,0,0,0,0,0,0,0,0,0,0,0,0,0.000000,0
183257,0,0,0,0,0,3,0,1,0,0,0,1,0.147775,0


In [0]:
train.columns

Index(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11',
       'encoded'],
      dtype='object')

In [0]:
serie_size

12

In [0]:
last_month = serie_size - 1
Y_train_encoded = train['label']
train.drop('label', axis=1, inplace=True)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [0]:

X_train_encoded = train[['11', 'encoded']]

Y_valid_encoded = valid['label']
valid.drop('label', axis=1, inplace=True)
X_valid_encoded = valid[['11', 'encoded']]

print("Train set", X_train_encoded.shape)
print("Validation set", X_valid_encoded.shape)

Train set (200327, 2)
Validation set (22259, 2)


/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [0]:
mlp_model = Sequential()
mlp_model.add(Dense(10, kernel_initializer='glorot_normal', activation='relu', input_dim=X_train_encoded.shape[1]))
mlp_model.add(Dense(10, kernel_initializer='glorot_normal', activation='relu'))
mlp_model.add(Dense(1))
mlp_model.summary()

adam = optimizers.Adam(0.0001)
mlp_model.compile(loss='mse', optimizer=adam)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 10)                30        
_________________________________________________________________
dense_5 (Dense)              (None, 10)                110       
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 11        
Total params: 151
Trainable params: 151
Non-trainable params: 0
_________________________________________________________________


In [0]:
mlp_history = mlp_model.fit(X_train_encoded.values, Y_train_encoded.values, epochs=50, batch_size=64, validation_data=(X_valid_encoded, Y_valid_encoded), verbose=2)

Train on 200327 samples, validate on 22259 samples
Epoch 1/50
 - 8s - loss: 1.2290 - val_loss: 1.1923
Epoch 2/50
 - 8s - loss: 1.2289 - val_loss: 1.1918
Epoch 3/50
 - 8s - loss: 1.2285 - val_loss: 1.1929
Epoch 4/50
 - 8s - loss: 1.2286 - val_loss: 1.1918
Epoch 5/50
 - 8s - loss: 1.2283 - val_loss: 1.1917
Epoch 6/50
 - 8s - loss: 1.2281 - val_loss: 1.1908
Epoch 7/50
 - 8s - loss: 1.2279 - val_loss: 1.1920
Epoch 8/50
 - 8s - loss: 1.2279 - val_loss: 1.1909
Epoch 9/50
 - 8s - loss: 1.2279 - val_loss: 1.1908
Epoch 10/50
 - 8s - loss: 1.2278 - val_loss: 1.1922
Epoch 11/50
 - 8s - loss: 1.2276 - val_loss: 1.1920
Epoch 12/50
 - 8s - loss: 1.2275 - val_loss: 1.1906
Epoch 13/50
 - 8s - loss: 1.2273 - val_loss: 1.1914
Epoch 14/50
 - 8s - loss: 1.2272 - val_loss: 1.1906
Epoch 15/50
 - 8s - loss: 1.2271 - val_loss: 1.1904
Epoch 16/50
 - 8s - loss: 1.2270 - val_loss: 1.1905
Epoch 17/50
 - 8s - loss: 1.2268 - val_loss: 1.1900
Epoch 18/50
 - 8s - loss: 1.2270 - val_loss: 1.1902
Epoch 19/50
 - 8s - lo

In [0]:
monthly_series=pd.read_csv('./monthly_series.csv')
test=pd.read_csv('./test.csv')
latest_records = monthly_series.drop_duplicates(subset=['shop_id', 'item_id'])
X_test = pd.merge(test, latest_records, on=['shop_id', 'item_id'], how='left', suffixes=['', '_'])
X_test.fillna(0, inplace=True)
X_test.drop(['ID', 'item_id', 'shop_id'], axis=1, inplace=True)
X_test.head()

,Unnamed: 0,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33
0,7131.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,2.0,2.0,0.0,0.0,0.0,1.0,1.0,1.0,3.0,1.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,7145.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,2.0,0.0,1.0,3.0,1.0
3,7144.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [0]:
X_test = X_test[['{}'.format(i + (34 - serie_size)) for i in range(serie_size)]]
X_test.head()

,22,23,24,25,26,27,28,29,30,31,32,33
0,1.0,2.0,2.0,0.0,0.0,0.0,1.0,1.0,1.0,3.0,1.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,3.0,2.0,0.0,1.0,3.0,1.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [0]:
X_test_reshaped = X_test.values.reshape((X_test.shape[0], X_test.shape[1], 1))
print(X_test_reshaped.shape)

(214200, 12, 1)


In [0]:
lstm_test_pred = lstm_model.predict(X_test_reshaped)

In [0]:
test_encoded = encoder.predict(X_test_reshaped)

In [0]:
X_test['encoded'] = test_encoded
X_test.head()

,22,23,24,25,26,27,28,29,30,31,32,33,encoded
0,1.0,2.0,2.0,0.0,0.0,0.0,1.0,1.0,1.0,3.0,1.0,0.0,0.357384
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
2,0.0,0.0,0.0,0.0,0.0,0.0,3.0,2.0,0.0,1.0,3.0,1.0,0.144394
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.000000
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000


In [0]:
X_test_encoded = X_test[['33', 'encoded']]
print("Train set", X_test_encoded.shape)
X_test_encoded.head()

Train set (214200, 2)


,33,encoded
0,0.0,0.357384
1,0.0,0.000000
2,1.0,0.144394
3,0.0,0.000000
4,0.0,0.000000


In [0]:
mlp_test_pred = mlp_model.predict(X_test_encoded)

In [0]:
lstm_prediction = pd.DataFrame(test['ID'], columns=['ID'])
lstm_prediction['item_cnt_month'] = lstm_test_pred.clip(0., 20.)
lstm_prediction.to_csv('lstm_submission.csv', index=False)
lstm_prediction.head(10)

,ID,item_cnt_month
0,0,0.450529
1,1,0.450529
2,2,0.450529
3,3,0.450529
4,4,0.450529
5,5,0.450529
6,6,0.450529
7,7,0.450529
8,8,0.450529
9,9,0.450529


In [0]:
mlp_prediction = pd.DataFrame(test['ID'], columns=['ID'])
mlp_prediction['item_cnt_month'] = mlp_test_pred.clip(0., 20.)
mlp_prediction.to_csv('mlp_submission1.csv', index=False)
mlp_prediction.head(10)

,ID,item_cnt_month
0,0,0.256854
1,1,0.286959
2,2,0.333082
3,3,0.286959
4,4,0.286959
5,5,0.461483
6,6,0.825796
7,7,0.281743
8,8,0.703687
9,9,0.286959


In [0]:
# Run this cell to mount your Google Drive.
from google.colab import drive
drive.mount('/content/drive')